In [1]:
import math
import numpy as np 

In [2]:
class decoder:
    
    def __init__(self,sig1,sig2):
        self.sig1=sig1
        self.sig2=sig2
        
    def binary_converter(self,sig):
        res = "{0:08b}".format(int(sig, 16))
        return res
    
    def decimal_converter(self,n):
        return int(n,2) 
    
    def NL(self,lat):
        NZ=15
        val=math.floor((2*math.pi)/(np.arccos(1-((1-np.cos(math.pi/(2*NZ)))/(np.cos(math.pi *lat/180))**2))))
        return val
   
    def breaking_down_the_signal(self):
        binary_string=self.binary_converter(self.sig1)
        Downlink_Format=binary_string[0:5]
        Capability =binary_string[5:8]
        ICAO_aircraft_address=binary_string[8:32]
        Type_code=binary_string[32:37]
        Data=binary_string[37:88]
        Interrogator_ID=binary_string[88:]

        print ("ADS-B message:", current_loc)  
        print('The length of the signal is',len(binary_string),'bits')
        print("Downlink_Format:",Downlink_Format)
        print("Capability:",Capability)
        print("ICAO_aircraft_address:",ICAO_aircraft_address)
        print("Type_code:",Type_code)
        print("Data:",Data)
        print("Interrogator_ID:",Interrogator_ID)
    
    def cpr(self):
        code_1=self.binary_converter(self.sig1)
        code_2=self.binary_converter(self.sig2)
        cpr_code_1=code_1[53]
        cpr_code_2=code_2[53]

        if(cpr_code_1=='0'):
            cpr_lat_even=self.decimal_converter(code_1[54:71])/131072
            cpr_lon_even= self.decimal_converter(code_1[71:88])/131072
            cpr_lat_odd= self.decimal_converter(code_2[54:71])/131072
            cpr_lon_odd= self.decimal_converter(code_2[71:88])/131072

        elif(cpr_code_1=='1'):
            cpr_lat_odd= self.decimal_converter(code_1[54:71])/131072
            cpr_lon_odd= self.decimal_converter(code_1[71:88])/131072
            cpr_lat_even= self.decimal_converter(code_2[54:71])/131072
            cpr_lon_even= self.decimal_converter(code_2[71:88])/131072


        j=math.floor(59*(cpr_lat_even) - 60*(cpr_lat_odd)+(1/2))
        dlat_even= 360/60
        dlat_odd=360/59
        relative_lat_even=dlat_even*(j%60 + cpr_lat_even)
        relative_lat_odd=dlat_odd*(j%59 + cpr_lat_odd)

        if(relative_lat_even>=270):
            even_lat=relative_lat_even-360
        else:
            even_lat=relative_lat_even
        if(relative_lat_odd>=270):
            odd_lat=relative_lat_odd-360
        else: 
            odd_lat=relative_lat_odd
         

        if(even_lat>odd_lat): 
            ni=max(self.NL(even_lat),1) 
            dLon=360/ni
            m=math.floor(cpr_lon_even*(self.NL(even_lat)-1)-(cpr_lon_odd*self.NL(even_lat))+(1/2))
            longitude=dLon*(m%ni +cpr_lon_even)

        elif(even_lat<odd_lat):
            ni=max(self.NL(odd_lat)-1,1) 
            dLon=360/ni
            m=math.floor(cpr_lon_even*(self.NL(odd_lat)-1)-(cpr_lon_odd*self.NL(odd_lat))+(1/2))
            longitude=dLon*(m%ni +cpr_lon_odd)

        if(longitude>=180):
            longitude=longitude-360
        else:
            longitude=longitude
        
        print("The even latitude is :",even_lat)
        print("The odd latitude is :",odd_lat)
        print("The longitude  is :",longitude)
        
    def altitude(self):
        binary_string=self.binary_converter(self.sig1)
        Altitude=binary_string[40:52]
        q_bit=binary_string[47]
        if(q_bit=='1'):
            factor=25
        elif(q_bit=='0'):
            factor=100

        mod_altitude=Altitude[0:6]+Altitude[7:]
        final_alt=self.decimal_converter(mod_altitude)*factor-1000
        print("Altitude:",final_alt,'ft.')
    
    def ground_speed(self):
        binary_string=self.binary_converter(self.sig1)
        S_ew=binary_string[45]
        S_ns=binary_string[56]
        V_ew=binary_string[46:56]
        V_ns=binary_string[57:67]

        if(S_ew=='0'):
            V_we=self.decimal_converter(V_ew)-1
        elif(S_ew=='1'):
            V_we=-1*(self.decimal_converter(V_ew) -1)    

        if(S_ns=='0'):
            V_sn=self.decimal_converter(V_ns)-1
        elif(S_ns=='1'):
            V_sn=-1*(self.decimal_converter(V_ns) -1)     

        horizontal_velocity=np.sqrt(V_we**2+V_sn**2)              # horizontal_velocity
        h=np.arctan2(V_we,V_sn) * 360 / (2*np.pi)

        if(h<0):
            h=h+360
        else:
            h=h

        S_vr=binary_string[68]

        if(S_vr=='0'):
            direc="ascending"
        elif(S_vr=='1'):
            direc="descending"

        Vr=64*(self.decimal_converter(binary_string[69:78])-1) 
        VrSrc=binary_string[67]

        if(VrSrc=='0'):
            change="Baro-pressure altitude change rate"
        elif(VrSrc=='1'):
            change="Geometric altitude change rate"

        print("Ground speed:",horizontal_velocity,'(kt)')
       # print("Heading aircraft:",h)  # heading
        print("The aircraft is",direc ,"at a rate of ",Vr,' (',change,')')    
        
    def air_velocity(self):
        binary_string=self.binary_converter(self.sig1)
        S_hdg=binary_string[45]
        Hdg=binary_string[46:56]
        heading=self.decimal_converter(Hdg)*360/1024
        AS_t=binary_string[56]

        if(AS_t=='0'):
            strin="Indicated Airspeed (IAS)"
        elif(AS_t=='1'):
            strin="True Airspeed (TAS)" 

        AS=binary_string[57:67]
        print(strin,':',self.decimal_converter(AS),'knots')
    
    def info(self):
        self.cpr()
        self.altitude()
        self.ground_speed()
        self.air_velocity()

In [5]:
new_signal = "8D40621D58C382D690C8AC2863A7"          
previous_signal= "8D40621D58C386435CC412692AD6"      
p1=decoder(new_signal,previous_signal)

In [6]:
p1.info()  

The even latitude is : 52.2572021484375
The odd latitude is : 52.26578017412606
The longitude  is : 3.938912527901786
Altitude: 38000 ft.
Ground speed: 1132.294131398728 (kt)
The aircraft is ascending at a rate of  3136  ( Geometric altitude change rate )
True Airspeed (TAS) : 692 knots
